In [1]:
# Imports
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship


In [2]:
# Create Engine
engine = create_engine("sqlite:///soundcloud.sqlite")

In [3]:
# Declare a Base object here
Base = declarative_base()

In [4]:
# Define the ORM class for `allTimePlayCount`
class allTimePlayCount(Base):
    __tablename__ = 'allTimePlayCount'
    __table_args__ = {'extend_existing': True} 
    
    Genre = Column(Text, primary_key=True)
    allTimePlays = Column(Integer)
    
    def __repr__(self):
        return f"id={self.Genre}, name={self.allTimePlays}"


In [5]:
# Define the ORM class for `weeklyPlayCount`
class weeklyPlayCount(Base):
    
    __tablename__ = 'weeklyPlayCount'
    __table_args__ = {'extend_existing': True} 


    Genre = Column(Text, primary_key=True)
    weeklyPlays = Column(Integer)

    def __repr__(self):
        return f"id={self.Genre}, name={self.weeklyPlays}"


In [6]:
# Use `create_all` to create the tables
Base.metadata.create_all(engine)


In [7]:
# Verify that the table names exist in the database
engine.table_names()


['allTimePlayCount', 'weeklyPlayCount']

In [9]:
# Use Pandas to Bulk insert each CSV file into their appropriate table

def populate_table(engine, table, csvfile):
    """Populates a table from a Pandas DataFrame."""
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    
    # Orient='records' creates a list of data to write
    # http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
    data = df_of_data_to_insert.to_dict(orient='records')

    # Optional: Delete all rows in the table 
    conn.execute(table.delete())

    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)
    
# Call the function to insert the data for each table
populate_table(engine, weeklyPlayCount.__table__, 'outputs/allWeeklyPlays.csv')
populate_table(engine, allTimePlayCount.__table__, 'outputs/allTimePlays.csv')


In [10]:
# Use a basic query to validate that the data was inserted correctly for table `allTimePlayCount`
engine.execute("SELECT * FROM allTimePlayCount").fetchall()
#NOTE: NEED TO RENAME THIS TABLE FIELD

[('Hip-Hop', 1493885155),
 ('R&B', 1075967811),
 ('Electronic Dance Music (EDM)', 873200005),
 ('Electronic', 734744808),
 ('House', 587570334),
 ('Latin', 549290825),
 ('Pop', 456973925),
 ('Dubstep', 321688198),
 ('Rock', 280774629),
 ('Reggaeton', 277572177),
 ('Deep House', 262601830),
 ('Alternative Rock', 136228544),
 ('Country', 133531995),
 ('World Music', 127501479),
 ('Ambient', 116192917),
 ('Reggae', 110091195),
 ('Dance Hall', 105870453),
 ('Classical', 94202177),
 ('SoundTrack', 93998209),
 ('Folk', 77611061),
 ('Indie', 77533317),
 ('Metal', 73225016),
 ('Piano', 56291433),
 ('Drum Bass', 47498811),
 ('Jazz', 45360087),
 ('Techno', 25895147),
 ('Disco', 15462207)]

In [11]:
# Use a basic query to validate that the data was inserted correctly for table `weeklyPlayCount`
engine.execute("SELECT * FROM weeklyPlayCount").fetchall()


[('Hip-Hop', 44035845),
 ('R&B', 9611441),
 ('Pop', 3679753),
 ('Latin', 1872466),
 ('Electronic', 1494733),
 ('Country', 1435120),
 ('Reggaeton', 1369026),
 ('Electronic Dance Music (EDM)', 1296888),
 ('World Music', 1234752),
 ('Rock', 978367),
 ('House', 636750),
 ('Dubstep', 630263),
 ('Alternative Rock', 389283),
 ('Folk', 330228),
 ('Dance Hall', 301825),
 ('SoundTrack', 273283),
 ('Ambient', 271726),
 ('Classical', 256497),
 ('Indie', 197182),
 ('Deep House', 190286),
 ('Metal', 152111),
 ('Reggae', 145850),
 ('Drum Bass', 104500),
 ('Jazz', 89526),
 ('Piano', 63727),
 ('Techno', 60992),
 ('Disco', 24476)]